# Geolocalización

En esta notebook haremos un intento de geolocalización con los textos de los usuarios.

Pero haremos algo distinto: usaremos Term Frequency - Inverse Province Frequency (TF-IPF)


[Geolocation prediction in social media data by finding location indicative words](http://www.aclweb.org/anthology/C12-1064)

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient

client = MongoClient('localhost', 27018)

db = client['contrastes']

In [2]:
import pandas as pd

df_train = pd.read_json("data/geoloc/users_train.json")
df_test = pd.read_json("data/geoloc/users_test.json")

Hagamos lo siguiente:

- Entrenemos con unigramas una regresión logística para 
- Luego probemos con los regionalismos

Primero, partamos en train, test

In [3]:
df_train.groupby("provincia").count()


,text
provincia,
buenosaires,337
catamarca,341
chaco,331
chubut,328
cordoba,317
corrientes,345
entrerios,338
formosa,286
jujuy,339


## Palabras precalculadas

Carguemos antes las palabras que sabemos que ocurren una cantidad razonable de veces

In [4]:
%%time
from contrastes.processing import build_dataframe_from_users

word_df = build_dataframe_from_users(row for index, row in df_train.iterrows())

CPU times: user 8min 3s, sys: 544 ms, total: 8min 3s
Wall time: 8min 3s


In [5]:
from contrastes.processing import preprocess_raw_df

word_df = preprocess_raw_df(word_df, filter_words=(10, 2))

/home/jmperez/projects/contrastes/notebooks/contrastes/processing.py:185: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_palabras = cant_palabras
/home/jmperez/projects/contrastes/notebooks/contrastes/processing.py:186: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.columnas_personas = cant_personas


In [25]:
%%time
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords


tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 27.2 µs


In [9]:
print("Vocabulario del vectorizador: {} palabras".format(len(vectorizer.vocabulary_)))

Vocabulario del vectorizador: 103783 palabras


In [26]:
from sklearn.preprocessing import LabelEncoder

province_encoder = LabelEncoder()

province_encoder.fit(df_train["provincia"].values)

LabelEncoder()

In [27]:
y_train = province_encoder.transform(df_train["provincia"].values)
y_test = province_encoder.transform(df_test["provincia"].values)

La reg. logística será un softmax, así que elijo `multi_class='multinomial'`

38% de accuracy

## Usando sólo "regionalismos" o LIW (Location Indicative Words)

Usemos ahora nuestros "features". Es decir, probemos con porcentajes de las palabras encontradas

In [28]:
from contrastes.lists import add_ival

word_df.columns

Index(['buenosaires_ocurrencias', 'buenosaires_usuarios',
       'catamarca_ocurrencias', 'catamarca_usuarios', 'chaco_ocurrencias',
       'chaco_usuarios', 'chubut_ocurrencias', 'chubut_usuarios',
       'cordoba_ocurrencias', 'cordoba_usuarios', 'corrientes_ocurrencias',
       'corrientes_usuarios', 'entrerios_ocurrencias', 'entrerios_usuarios',
       'formosa_ocurrencias', 'formosa_usuarios', 'jujuy_ocurrencias',
       'jujuy_usuarios', 'lapampa_ocurrencias', 'lapampa_usuarios',
       'larioja_ocurrencias', 'larioja_usuarios', 'mendoza_ocurrencias',
       'mendoza_usuarios', 'misiones_ocurrencias', 'misiones_usuarios',
       'neuquen_ocurrencias', 'neuquen_usuarios', 'rionegro_ocurrencias',
       'rionegro_usuarios', 'salta_ocurrencias', 'salta_usuarios',
       'sanjuan_ocurrencias', 'sanjuan_usuarios', 'sanluis_ocurrencias',
       'sanluis_usuarios', 'santacruz_ocurrencias', 'santacruz_usuarios',
       'santafe_ocurrencias', 'santafe_usuarios', 'santiago_ocurrencias',
  

In [31]:
word_df.sort_values(["cant_provincias", "cant_palabra"], ascending=[True, False], inplace=True)

word_df.iloc[:10][["cant_palabra", "cant_provincias"]]

,cant_palabra,cant_provincias
tiemposur,883.0,1
logroño,711.0,1
nihuil,450.0,1
chivil,332.0,1
ipauss,315.0,1
vallerga,291.0,1
asprodema,290.0,1
cdelu,244.0,1
calahorra,216.0,1
canicross,202.0,1


Veamos qué performance tiene usando 1000, 2000, 3000, y así...

In [34]:
%%time 

liw_vectorizer = CountVectorizer(
    tokenizer=tokenizer.tokenize,
    vocabulary=word_df.index)

X_train = liw_vectorizer.fit_transform(df_train["text"])
print("Vectorizing")
X_test = liw_vectorizer.transform(df_test["text"])

Vectorizing
CPU times: user 10min 3s, sys: 448 ms, total: 10min 3s
Wall time: 10min 3s


Ya las tenemos vectorizadas en el orden esperado!

In [35]:

clfs = {}
scores = {}

In [39]:
from sklearn.linear_model import LogisticRegression


for num_words in range(250, 5000, 250):
    if num_words in clfs:
        print("{} palabras ----> accuracy {:.2f}".format(num_words, scores[num_words]*100))
        continue
    X_tr = X_train[:, :num_words].todense()
    X_tst = X_test[:, :num_words].todense()
    
    clf = LogisticRegression(
        multi_class='multinomial', solver='saga', penalty='l2', 
        max_iter=200, n_jobs=-1)
    clf.fit(X_tr, y_train)
    
    scores[num_words] = clf.score(X_tst, y_test)
    print("{} palabras ----> accuracy {:.2f}".format(num_words, scores[num_words]*100))
    clfs[num_words] = clf
    

250 palabras ----> accuracy 21.88
500 palabras ----> accuracy 27.48
750 palabras ----> accuracy 30.00
1000 palabras ----> accuracy 31.56
1250 palabras ----> accuracy 36.76
1500 palabras ----> accuracy 41.80
1750 palabras ----> accuracy 43.28
2000 palabras ----> accuracy 43.84
2250 palabras ----> accuracy 44.28
2500 palabras ----> accuracy 45.12
2750 palabras ----> accuracy 49.16
3000 palabras ----> accuracy 52.48
3250 palabras ----> accuracy 53.76
3500 palabras ----> accuracy 53.84
3750 palabras ----> accuracy 54.28
4000 palabras ----> accuracy 54.24
4250 palabras ----> accuracy 56.76
4500 palabras ----> accuracy 57.56
4750 palabras ----> accuracy 58.16


2500 palabras dan un accuracy de 71%. BASTANTE BIEN. Luego disminuye la performance

In [42]:
for num_words in range(5000, 20000, 500):
    if num_words in clfs:
        print("{} palabras ----> accuracy {:.2f}".format(num_words, scores[num_words]*100))
        continue
    X_tr = X_train[:, :num_words].todense()
    X_tst = X_test[:, :num_words].todense()
    
    clf = LogisticRegression(
        multi_class='multinomial', solver='saga', penalty='l2', 
        max_iter=200, n_jobs=-1)
    clf.fit(X_tr, y_train)
    
    scores[num_words] = clf.score(X_tst, y_test)
    print("{} palabras ----> accuracy {:.2f}".format(num_words, scores[num_words]*100))
    clfs[num_words] = clf

5000 palabras ----> accuracy 58.04
5500 palabras ----> accuracy 58.24
6000 palabras ----> accuracy 60.76
6500 palabras ----> accuracy 61.56
7000 palabras ----> accuracy 61.60
7500 palabras ----> accuracy 61.64
8000 palabras ----> accuracy 61.92
8500 palabras ----> accuracy 64.08
9000 palabras ----> accuracy 64.28
9500 palabras ----> accuracy 64.44
10000 palabras ----> accuracy 64.36
10500 palabras ----> accuracy 64.52
11000 palabras ----> accuracy 64.60
11500 palabras ----> accuracy 65.60
12000 palabras ----> accuracy 65.84
12500 palabras ----> accuracy 66.12
13000 palabras ----> accuracy 65.96
13500 palabras ----> accuracy 65.92
14000 palabras ----> accuracy 66.00
14500 palabras ----> accuracy 66.04
15000 palabras ----> accuracy 66.08
15500 palabras ----> accuracy 66.04
16000 palabras ----> accuracy 66.08
16500 palabras ----> accuracy 67.00


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


17000 palabras ----> accuracy 67.16


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


17500 palabras ----> accuracy 67.04


...

18500 palabras ----> accuracy 66.96


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


19000 palabras ----> accuracy 67.04


/home/jmperez/.pyenv/versions/3.6.5/envs/contrastes/lib/python3.6/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


19500 palabras ----> accuracy 67.08


In [43]:
import pickle

pickle.dump(clfs, open("clfs_ipf.pkl", "wb"))
